### Verizon RFDS PDF Ripping
#### Azure Computer Vision Trial
Test using Azure Computer Vision OCR APIs to process verizon RF Data Sheets.  The goal is to identify the carrier (Verizon), Tower Longitude, Tower Latitude, and ENodeB ID.

##### Libraries Installed
conda install -c conda-forge azure-common

pip install azure-cognitiveservices-vision-computervision

conda install pillow

pip install azure-storage-blob

pip install azure-mgmt-storage

##### Helpful Links
https://github.com/Azure-Samples/cognitive-services-quickstart-code/tree/master/python/ComputerVision

https://docs.microsoft.com/en-us/azure/cognitive-services/computer-vision/quickstarts-sdk/client-library?tabs=visual-studio&pivots=programming-language-python

https://docs.microsoft.com/en-us/python/api/overview/azure/storage-blob-readme?view=azure-python

https://docs.microsoft.com/en-us/azure/cognitive-services/computer-vision/vision-api-how-to-topics/call-read-api

https://github.com/Azure-Samples/cognitive-services-quickstart-code/blob/master/python/ComputerVision/ExtractTextPDF.py

In [8]:
### Import Libraries
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

import pandas as pd
#from array import array
#import os
#from PIL import Image
import sys
import time

In [2]:
### Azure Computer Vision Keys
subscription_key = "41e484a5197d4ca0b79af7023e5c1fdd"
endpoint = "https://cci-ds-compvis-nonprod.cognitiveservices.azure.com/"
region = "eastus2"

In [3]:
### Authenticate the computer vision client
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))
print(computervision_client.api_version)

3.2


In [31]:
### Get RFDS from blob storage
#import os, uuid
#from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, __version__

## Create Blob Storage Client
#try:
#    blob_service = BlobServiceClient(account_url="https://storageaccountccids99ed.blob.core.windows.net/", credential="nSK3rxAVPFkhJfV6qfJqdScvHSv9YRKGpGkBPSWgYO4fgUlsk9E1WXxOU+07+KRn7BdLSAvFp6N5CNLSMqZN5Q==")
#    print("Blob Service API Version: ", blob_service.api_version)
#except Exception as ex:
#    print("Blob Client Exception: ", ex)

Blob Service API Version:  2020-10-02


In [4]:
### List Blobs in a Container

from azure.storage.blob import ContainerClient, BlobClient

cn = "cci-ds-rfds-vzw"
cs = "DefaultEndpointsProtocol=https;AccountName=storageaccountccids99ed;AccountKey=nSK3rxAVPFkhJfV6qfJqdScvHSv9YRKGpGkBPSWgYO4fgUlsk9E1WXxOU+07+KRn7BdLSAvFp6N5CNLSMqZN5Q==;EndpointSuffix=core.windows.net"

try:
    container = ContainerClient.from_connection_string(conn_str=cs, container_name=cn)
    print("Container connection successful")
except Exception as e:
    print("Container connection failed: ", e)

try:
    blob_list = container.list_blobs()
    for blob in blob_list:
        print(blob.name + '\n')
        full_blob = BlobClient.from_connection_string(conn_str=cs, container_name=cn, blob_name=blob.name)
        #full_blob_data = full_blob.download_blob()
except Exception as e:
    print("Blob listing failed: ", e)


Container connection successful
REDLINES_880315_AIR_TRAFFIC_CONTROL_Verizon_LTE_PCD_rev_A_08.01.2019.pdf

Verizon__Satsuma_South__BU_813007.pdf



In [5]:
full_blob.url

'https://storageaccountccids99ed.blob.core.windows.net/cci-ds-rfds-vzw/Verizon__Satsuma_South__BU_813007.pdf'

In [9]:
### Call the ACV Read API
prev_line = ""
lat = ""
long = ""
enodeb = ""
carrier = ""
rfds_df = pd.DataFrame(columns= ['Carrier', 'Latitude', 'Longitude', 'EnodeB'])

try:
    #Call Read API
    cv_response = computervision_client.read(full_blob.url, raw=True)
    operation_location = cv_response.headers["Operation-Location"]
    operation_id = operation_location.split("/")[-1]
    ###time.sleep(60)
    ###read_result = computervision_client.get_read_result(operation_id)
    # allow computer vision time to get results
    while True:
        read_result = computervision_client.get_read_result(operation_id)
        if read_result.status.lower() not in ['notstarted', 'running']:
            break
        print("Waiting for Computer Vision Read API to complete text extraction...")
        time.sleep(15)
    #if computer vision successfull start finding key components of output
    if (read_result.status.lower() == 'succeeded'):
        for r in read_result.analyze_result.read_results:
            for line in r.lines:
                if ("verizon" in prev_line and len(carrier) == 0):
                    carrier = "VZW"
                    print("Carrier = Verizon")
                elif ("latitude" in prev_line):
                    lat = line.text
                    print("Latitude = ", lat)
                elif ("longitude" in prev_line):
                    long = line.text
                    print("Longitude = ", long)
                elif ("nodeb" in prev_line):
                    enodeb = line.text
                    print("EnodeB = ", enodeb)
                elif ("nedeb" in prev_line):
                    enodeb = line.text
                    print("EnodeB = ", enodeb)
                else:
                    prev_line = line.text.lower()

                if (carrier == "VZW" and len(lat) > 0 and len(long) > 0 and len(enodeb)):
                    rfds_df.loc[len(rfds_df.index)] = [carrier, lat, long, enodeb]
                    
                prev_line = line.text.lower()
except Exception as e:
    print("filepath exception: ", e)


Waiting for Computer Vision Read API to complete text extraction...
Carrier = Verizon
EnodeB =  127252
Latitude =  30.834861 / 30 50' 5.49" N
Longitude =  -88.055278 / 88 3' 19.0" W


In [10]:
rfds_df.loc[len(rfds_df.index)] = [carrier, lat, long, enodeb]

In [81]:
prev_line = ""
lat = ""
long = ""
enodeb = ""
carrier = ""

for r in read_result.analyze_result.read_results:
    for line in r.lines:
        if ("verizon" in prev_line and len(carrier) == 0):
            carrier = "VZW"
            print("Carrier = Verizon")
        elif ("latitude" in prev_line):
            lat = line.text
            print("Latitude = ", lat)
        elif ("longitude" in prev_line):
            long = line.text
            print("Longitude = ", long)
        elif ("nodeb" in prev_line):
            enodeb = line.text
            print("EnodeB = ", enodeb)
        elif ("nedeb" in prev_line):
            enodeb = line.text
            print("EnodeB = ", enodeb)
        else:
            prev_line = line.text.lower()
            
        prev_line = line.text.lower()
        #print(line.text)
        #print(line.bounding_box)

Carrier = Verizon
EnodeB =  127252
Latitude =  30.834861 / 30 50' 5.49" N
Longitude =  -88.055278 / 88 3' 19.0" W
